In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

import os
from utils import *
from settings import station_locale, data_root, output_folder, report_folder, station_dist, station_angle, station_locale

from sklearn.externals import joblib
from sklearn.model_selection import train_test_split 

from xgboost import XGBRegressor

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder

In [3]:
data_folder = '../outputs/'
report_folder = '../report_rc/'
model_folder = '../models/'

## Construct test set

In [ ]:
gwt = pd.read_csv(os.path.join(data_folder, 'grid_weather_test.zip'))
owt = pd.read_csv(os.path.join(data_folder, 'ob_weather_test.zip'))

In [ ]:
useful_cols = ['longitude', 'latitude', 'local_time',
               'temperature', 'pressure', 'humidity',
               'wind_direction', 'wind_speed']

In [ ]:
df_indicator = gwt[useful_cols]
df_indicator = df_indicator.append(owt[useful_cols])

In [ ]:
df_indicator.head()

In [8]:
gwt.head()

Unnamed: 0        station_id  longitude  latitude           local_time  \
0           0  beijing_grid_000      115.0      39.0  2018-05-01 08:00:00   
1           1  beijing_grid_001      115.0      39.1  2018-05-01 08:00:00   
2           2  beijing_grid_002      115.0      39.2  2018-05-01 08:00:00   
3           3  beijing_grid_003      115.0      39.3  2018-05-01 08:00:00   
4           4  beijing_grid_004      115.0      39.4  2018-05-01 08:00:00   

   temperature  pressure  humidity  wind_direction  wind_speed weather  \
0         20.0  975.6105      52.0           76.64        4.38  CLOUDY   
1         20.0  962.0873      49.0           58.35        3.82  CLOUDY   
2         13.0  948.5641      46.0           36.82        3.75  CLOUDY   
3         13.0  933.2105      43.0            9.52        4.65  CLOUDY   
4         13.0  916.0265      42.0          350.17        6.88  CLOUDY   

         date      time  solar_term  time_label  datetime_label  
0  2018-05-01  08:00:00          15           6               0  
1  2018-05-01  08:00:00          15           6               0  
2  2018-05-01  08:00:00          15           6               0  
3  2018-05-01  08:00:00          15           6               0  
4  2018-05-01  08:00:00          15           6               0

In [ ]:
station_list = ['aotizhongxin_aq', 'badaling_aq', 'beibuxinqu_aq', 'daxing_aq',
                'dingling_aq', 'donggaocun_aq', 'dongsi_aq', 'dongsihuan_aq',
                'fangshan_aq', 'fengtaihuayuan_aq', 'guanyuan_aq', 'gucheng_aq',
                'huairou_aq', 'liulihe_aq', 'mentougou_aq', 'miyun_aq',
                'miyunshuiku_aq', 'nansanhuan_aq', 'nongzhanguan_aq',
                'pingchang_aq', 'pinggu_aq', 'qianmen_aq', 'shunyi_aq',
                'tiantan_aq', 'tongzhou_aq', 'wanliu_aq', 'wanshouxigong_aq',
                'xizhimenbei_aq', 'yanqin_aq', 'yizhuang_aq', 'yongdingmennei_aq',
                'yongledian_aq', 'yufa_aq', 'yungang_aq', 'zhiwuyuan_aq']
localtimes = gwt['local_time'].unique()

In [ ]:
test_df = pd.DataFrame()
for s in station_list:
    d = {}
    d['station_id'] = [s for i in range(48)]
    d['test_id'] = [f'{s}#{i}' for i in range(48)]
    d['local_time'] = localtimes
    locale_info = station_locale[s]
    d['longitude'] = [locale_info[0][0] for i in range(48)]
    d['latitude'] = [locale_info[0][1] for i in range(48)]
    d['area'] = [locale_info[1] for i in range(48)]
    df = pd.DataFrame(d)
    test_df = test_df.append(df)

In [ ]:
test_df['date'] = test_df['local_time'].map(lambda x: x.split()[0])
test_df['time'] = test_df['local_time'].map(lambda x: x.split()[1])
test_df['periodic_solar_term'] = test_df['date'].map(lambda x: get_solar_term(x, periodicity=True))
test_df['periodic_time'] = test_df['time'].map(lambda x: get_periodic_time_label(x))
test_df = test_df.drop(['area'], axis=1).join(pd.get_dummies(test_df['area']))

In [ ]:
gw_df = pd.read_csv(os.path.join(data_folder, 'grid_weather_processed.zip'))
ow_df = pd.read_csv(os.path.join(data_folder, 'ob_weather_processded.zip'))
df_indicator_old = gw_df[useful_cols]
df_indicator_old = df_indicator.append(ow_df[useful_cols])

In [13]:
indicators = gw_df[useful_cols].append(ow_df[useful_cols]).query('local_time > "2018-04-28 07:00:00"')

In [14]:
del df_indicator_old
del gw_df
del ow_df

In [15]:
indicators = indicators.append(df_indicator)

In [16]:
datetime_labeler = LabelEncoder()
indicators['datetime_label'] = datetime_labeler.fit_transform(indicators['local_time'])
test_df['datetime_label'] = datetime_labeler.transform(test_df['local_time'])

In [17]:
indicators.head()

longitude  latitude           local_time  temperature  pressure  \
7451934      115.0      39.0  2018-04-28 08:00:00         17.0  972.9408   
7451935      115.0      39.1  2018-04-28 08:00:00         17.0  959.5904   
7451936      115.0      39.2  2018-04-28 08:00:00         13.0  946.2400   
7451937      115.0      39.3  2018-04-28 08:00:00         13.0  931.1008   
7451938      115.0      39.4  2018-04-28 08:00:00         13.0  914.1728   

         humidity  wind_direction  wind_speed  datetime_label  
7451934      51.0          278.91        2.79               0  
7451935      48.0          304.32        2.43               0  
7451936      46.0          331.43        2.62               0  
7451937      43.0          336.54        3.48               0  
7451938      40.0          329.27        4.68               0

In [18]:
idct_list = ['temperature','pressure','humidity','wind_direction','wind_speed']
for indic in idct_list:
    neibours = 7 if indic in ['temperature','pressure','humidity'] else 5
    weight = 'distance' # 'uniform', 'distance'

    train_x = indicators[['longitude', 'latitude', 'datetime_label']]
    train_y = indicators[indic]
    knn = KNeighborsRegressor(n_neighbors=neibours, weights=weight)
    knn.fit(train_x, train_y)
    test_df[indic] = knn.predict(test_df[['longitude', 'latitude', 'datetime_label']])

In [19]:
test_df = test_df.drop_duplicates(subset=['test_id'], keep='first')

In [20]:
test_df

station_id               test_id           local_time  longitude  \
0     aotizhongxin_aq     aotizhongxin_aq#0  2018-05-01 08:00:00    116.397   
0         badaling_aq         badaling_aq#0  2018-05-01 08:00:00    115.988   
0       beibuxinqu_aq       beibuxinqu_aq#0  2018-05-01 08:00:00    116.174   
0           daxing_aq           daxing_aq#0  2018-05-01 08:00:00    116.404   
0         dingling_aq         dingling_aq#0  2018-05-01 08:00:00    116.220   
0       donggaocun_aq       donggaocun_aq#0  2018-05-01 08:00:00    117.120   
0           dongsi_aq           dongsi_aq#0  2018-05-01 08:00:00    116.417   
0       dongsihuan_aq       dongsihuan_aq#0  2018-05-01 08:00:00    116.483   
0         fangshan_aq         fangshan_aq#0  2018-05-01 08:00:00    116.136   
0   fengtaihuayuan_aq   fengtaihuayuan_aq#0  2018-05-01 08:00:00    116.279   
0         guanyuan_aq         guanyuan_aq#0  2018-05-01 08:00:00    116.339   
0          gucheng_aq          gucheng_aq#0  2018-05-01 08:00:00    116.184   
0          huairou_aq          huairou_aq#0  2018-05-01 08:00:00    116.628   
0          liulihe_aq          liulihe_aq#0  2018-05-01 08:00:00    116.000   
0        mentougou_aq        mentougou_aq#0  2018-05-01 08:00:00    116.106   
0            miyun_aq            miyun_aq#0  2018-05-01 08:00:00    116.832   
0      miyunshuiku_aq      miyunshuiku_aq#0  2018-05-01 08:00:00    116.911   
0       nansanhuan_aq       nansanhuan_aq#0  2018-05-01 08:00:00    116.368   
0     nongzhanguan_aq     nongzhanguan_aq#0  2018-05-01 08:00:00    116.461   
0        pingchang_aq        pingchang_aq#0  2018-05-01 08:00:00    116.230   
0           pinggu_aq           pinggu_aq#0  2018-05-01 08:00:00    117.100   
0          qianmen_aq          qianmen_aq#0  2018-05-01 08:00:00    116.395   
0           shunyi_aq           shunyi_aq#0  2018-05-01 08:00:00    116.655   
0          tiantan_aq          tiantan_aq#0  2018-05-01 08:00:00    116.407   
0         tongzhou_aq         tongzhou_aq#0  2018-05-01 08:00:00    116.663   
0           wanliu_aq           wanliu_aq#0  2018-05-01 08:00:00    116.287   
0    wanshouxigong_aq    wanshouxigong_aq#0  2018-05-01 08:00:00    116.352   
0      xizhimenbei_aq      xizhimenbei_aq#0  2018-05-01 08:00:00    116.349   
0           yanqin_aq           yanqin_aq#0  2018-05-01 08:00:00    115.972   
0         yizhuang_aq         yizhuang_aq#0  2018-05-01 08:00:00    116.506   
..                ...                   ...                  ...        ...   
47      donggaocun_aq      donggaocun_aq#47  2018-05-03 07:00:00    117.120   
47          dongsi_aq          dongsi_aq#47  2018-05-03 07:00:00    116.417   
47      dongsihuan_aq      dongsihuan_aq#47  2018-05-03 07:00:00    116.483   
47        fangshan_aq        fangshan_aq#47  2018-05-03 07:00:00    116.136   
47  fengtaihuayuan_aq  fengtaihuayuan_aq#47  2018-05-03 07:00:00    116.279   
47        guanyuan_aq        guanyuan_aq#47  2018-05-03 07:00:00    116.339   
47         gucheng_aq         gucheng_aq#47  2018-05-03 07:00:00    116.184   
47         huairou_aq         huairou_aq#47  2018-05-03 07:00:00    116.628   
47         liulihe_aq         liulihe_aq#47  2018-05-03 07:00:00    116.000   
47       mentougou_aq       mentougou_aq#47  2018-05-03 07:00:00    116.106   
47           miyun_aq           miyun_aq#47  2018-05-03 07:00:00    116.832   
47     miyunshuiku_aq     miyunshuiku_aq#47  2018-05-03 07:00:00    116.911   
47      nansanhuan_aq      nansanhuan_aq#47  2018-05-03 07:00:00    116.368   
47    nongzhanguan_aq    nongzhanguan_aq#47  2018-05-03 07:00:00    116.461   
47       pingchang_aq       pingchang_aq#47  2018-05-03 07:00:00    116.230   
47          pinggu_aq          pinggu_aq#47  2018-05-03 07:00:00    117.100   
47         qianmen_aq         qianmen_aq#47  2018-05-03 07:00:00    116.395   
47          shunyi_aq          shunyi_aq#47  2018-05-03 07:00:00    116.655   
47         tiantan_aq         tiantan_aq#47  2018-05-03 07:00

In [21]:
indicator = ['temperature', 'pressure', 'humidity', 'wind_speed', 'wind_direction']
temp = test_df.query('station_id=="aotizhongxin_aq"')

plt.figure(figsize=(4,8), dpi=200)
i = 1
for indic in indicator:
    plt.subplot(5,1,i)
    plt.plot(temp['datetime_label'], temp[indic])
    plt.title(f'aotizhongxin {indic}')
    plt.ylabel(indic)
    plt.xlabel('datetime_label')
    plt.xticks(rotation=90)
    i+=1
plt.tight_layout()
plt.savefig(os.path.join(report_folder, 'atzx_predict_weather_indocators.png'))
plt.show()

In [24]:
output_df = pd.DataFrame()
for target_station in station_list:
    station_name = target_station.split('_')[0]
    input_x = construct_train_xy(test_df, target_station, station_list, only_x=True)
    output = input_x[['station_id', 'test_id']]
    input_x = input_x.drop(['station_id', 'test_id'], axis=1)
    m_o3_1 = joblib.load(os.path.join(model_folder, f'{station_name}_o3_2.joblib'))
    m_o3_2 = joblib.load(os.path.join(model_folder, f'{station_name}_o3_3.joblib'))
    m_pm10_1 = joblib.load(os.path.join(model_folder, f'{station_name}_pm10_1.joblib'))
    m_pm25_1 = joblib.load(os.path.join(model_folder, f'{station_name}_pm25_1.joblib'))
    m_pm25_2 = joblib.load(os.path.join(model_folder, f'{station_name}_pm25_2.joblib'))
    m_pm25_3 = joblib.load(os.path.join(model_folder, f'{station_name}_pm25_3.joblib'))
    
    output['O3'] = (m_o3_1.predict(input_x)+m_o3_2.predict(input_x))/2
    output['PM2.5'] = (m_pm25_1.predict(input_x)+m_pm25_2.predict(input_x)+m_pm25_3.predict(input_x))/3
    output['PM10'] = m_pm10_1.predict(input_x)
    
    output_df = output_df.append(output)

# output_df = output_df.set_index('station_id')
output_df = output_df[['test_id', 'PM2.5', 'PM10', 'O3']]

In [30]:
output_df.to_csv(os.path.join(output_folder, 'submission.csv'), index = False)

In [27]:
# output_df

In [4]:
test_df = pd.read_csv(os.path.join(output_folder, 'test.zip'))

In [5]:
test_df

Unnamed: 0         station_id               test_id  \
0              0    aotizhongxin_aq     aotizhongxin_aq#0   
1              0        badaling_aq         badaling_aq#0   
2              0      beibuxinqu_aq       beibuxinqu_aq#0   
3              0          daxing_aq           daxing_aq#0   
4              0        dingling_aq         dingling_aq#0   
5              0      donggaocun_aq       donggaocun_aq#0   
6              0          dongsi_aq           dongsi_aq#0   
7              0      dongsihuan_aq       dongsihuan_aq#0   
8              0        fangshan_aq         fangshan_aq#0   
9              0  fengtaihuayuan_aq   fengtaihuayuan_aq#0   
10             0        guanyuan_aq         guanyuan_aq#0   
11             0         gucheng_aq          gucheng_aq#0   
12             0         huairou_aq          huairou_aq#0   
13             0         liulihe_aq          liulihe_aq#0   
14             0       mentougou_aq        mentougou_aq#0   
15             0           miyun_aq            miyun_aq#0   
16             0     miyunshuiku_aq      miyunshuiku_aq#0   
17             0      nansanhuan_aq       nansanhuan_aq#0   
18             0    nongzhanguan_aq     nongzhanguan_aq#0   
19             0       pingchang_aq        pingchang_aq#0   
20             0          pinggu_aq           pinggu_aq#0   
21             0         qianmen_aq          qianmen_aq#0   
22             0          shunyi_aq           shunyi_aq#0   
23             0         tiantan_aq          tiantan_aq#0   
24             0        tongzhou_aq         tongzhou_aq#0   
25             0          wanliu_aq           wanliu_aq#0   
26             0   wanshouxigong_aq    wanshouxigong_aq#0   
27             0     xizhimenbei_aq      xizhimenbei_aq#0   
28             0          yanqin_aq           yanqin_aq#0   
29             0        yizhuang_aq         yizhuang_aq#0   
...          ...                ...                   ...   
1650          47      donggaocun_aq      donggaocun_aq#47   
1651          47          dongsi_aq          dongsi_aq#47   
1652          47      dongsihuan_aq      dongsihuan_aq#47   
1653          47        fangshan_aq        fangshan_aq#47   
1654          47  fengtaihuayuan_aq  fengtaihuayuan_aq#47   
1655          47        guanyuan_aq        guanyuan_aq#47   
1656          47         gucheng_aq         gucheng_aq#47   
1657          47         huairou_aq         huairou_aq#47   
1658          47         liulihe_aq         liulihe_aq#47   
1659          47       mentougou_aq       mentougou_aq#47   
1660          47           miyun_aq           miyun_aq#47   
1661          47     miyunshuiku_aq     miyunshuiku_aq#47   
1662          47      nansanhuan_aq      nansanhuan_aq#47   
1663          47    nongzhanguan_aq    nongzhanguan_aq#47   
1664          47       pingchang_aq       pingchang_aq#47   
1665          47          pinggu_aq          pinggu_aq#47   
1666          47         qianmen_aq         qianmen_aq#47   
1667          47          shunyi_aq          shunyi_aq#47   
1668          47         tiantan_aq         tiantan_aq#47   
1669          47        tongzhou_aq        tongzhou_aq#47   
1670          47          wanliu_aq          wanliu_aq#47   
1671          47   wanshouxigong_aq   wanshouxigong_aq#47   
1672          47     xizhimenbei_aq     xizhimenbei_aq#47   
1673          47          yanqin_aq          yanqin_aq#47   
1674          47        yizhuang_aq        yizhuang_aq#47   
1675          47  yongdingmennei_aq  yongdingmennei_aq#47   
1676          47      yongledian_aq      yongledian_aq#47   
1677          47            yufa_aq            yufa_aq#47   
1678          47         yungang_aq         yungang_aq#47   
1679          47       zhiwuyuan_aq       zhiwuyuan_aq#47   

               local_time  longitude  latitude        date      time  \
0     2018-05-01 08:00:00    116.397    39.982  2018-05-01  08:00:00   
1     2018-05-01 08:00:00    115.988    40.365  2018-05-01  08:00:00   
2     2